In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.pipeline import FunctionTransformer
from tqdm import tqdm
import time

# Loading the dataset
df = pd.read_csv('../data/raw/ai4i2020.csv')

# Droping IDs (No predictive power)
df = df.drop(columns=['UDI', 'Product ID'], axis=1)
print(f"Raw Data Shape: {df.shape}")

Raw Data Shape: (10000, 12)


In [20]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()

        # 1. Physics-Informed Features
        # Power [W] = Torque [Nm] * Speed [rad/s] 
        # (Speed is in RPM, conversion factor is 2*pi/60)
        X['Power [W]'] = X['Torque [Nm]'] * (X['Rotational speed [rpm]'] * (2 * np.pi / 60))

        # Temperature Difference [K] (Process - Air)
        # Indicates heat dissipation efficiency
        X['Temp Diff [K]'] = X['Process temperature [K]'] - X['Air temperature [K]']

        # 2. Tool Wear Binning (Optional but helpful for trees)
        # 0-100 (New), 100-200 (Used), 200+ (Critical)
        X['Wear_Status'] = pd.cut(X['Tool wear [min]'],
                                  bins = [-1, 60, 180, 300],
                                  labels = [0, 1, 2] # Low, Med, High
                            ).astype(int)
        return X

# Testing it immediately
fe = FeatureEngineer()
df = fe.transform(df)
print(f"Data Shape after Feature Engineering: {df.shape}")
display(df.head())

Data Shape after Feature Engineering: (10000, 15)


,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Power [W],Temp Diff [K],Wear_Status
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,6951.590560,10.5,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,6826.722724,10.5,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,7749.387543,10.4,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,5927.504659,10.4,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,5897.816608,10.5,0


In [21]:
# Identifying Columns for Preprocessing Based on Types
NUMERIC_FEATURES = [
    'Air temperature [K]', 'Process temperature [K]', 
    'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]',
    'Power [W]', 'Temp Diff [K]', 'Wear_Status'
]
CATEGORICAL_FEATURES = ['Type']

In [22]:
# 1.Defining Candidate for Preprocessing Steps

scalers = {
    "StandardScaler" : StandardScaler(),
    "RobustScaler"   : RobustScaler()
}

encoders = {
    "OneHotEncoder" : OneHotEncoder(handle_unknown='ignore', sparse_output=False),
    "OrdinalEncoder" : OrdinalEncoder(categories=[['L', 'M', 'H']], dtype=int) # Type has 3 categories: 0,1,2 - mapped from 'L', 'M', 'H'
}

samplers = {
    "None" : None,
    "SMOTE" : SMOTE(random_state=42),
    "RandomOverSampler" : RandomOverSampler(random_state=42)
    # Note: Class Weights are handled inside the model, not as a sampler step
}

models = {
    "LogReg": LogisticRegression(max_iter=1000, random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=50, random_state=42)
}

In [23]:
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env file

CONFIG = {
"test_size": 0.2,
"random_state": 42,
"DAGSHUB_REPO_OWNER" : os.getenv("DagsHub_Repo_Owner"),
"DAGSHUB_REPO_NAME" : os.getenv("DagsHub_Repo_Name"),
"DAGSHUB_TRACKING_URI" : os.getenv("DagsHub_MLflow_Tracking_URI"),
"Experiment_Name": "Predictive_Maintenance_IIOT_Preprocessing",
}

In [24]:
import dagshub
import mlflow

# Initialize DagsHub MLflow Tracking

dagshub.init(repo_owner=CONFIG["DAGSHUB_REPO_OWNER"], repo_name=CONFIG["DAGSHUB_REPO_NAME"], mlflow=True)
mlflow.set_tracking_uri(CONFIG["DAGSHUB_TRACKING_URI"])
mlflow.set_experiment(CONFIG["Experiment_Name"])

Initialized MLflow to track repo "PrakashD2003/Smart-IIOT-Monitoring"

Repository PrakashD2003/Smart-IIOT-Monitoring initialized!

<Experiment: artifact_location='mlflow-artifacts:/510ed1c8c6994bc38b21e04f31bffb90', creation_time=1764967317151, experiment_id='0', last_update_time=1764967317151, lifecycle_stage='active', name='Predictive_Maintenance_IIOT_Preprocessing', tags={}>

In [25]:
def log_model_params(algo_name: str, model: BaseEstimator) -> None:
    """Logs model hyperparameters safely to MLflow."""
    
    if mlflow.active_run() is None:
        raise RuntimeError("No active MLflow run. Use mlflow.start_run().")
    
    try:
        print(f"Logging parameters for model: {algo_name}")

        params = model.get_params()

        # Convert non-primitive params to strings (MLflow requirement)
        clean_params = {k: str(v) if not isinstance(v, (int, float, str, bool)) else v
                        for k, v in params.items()}

        mlflow.log_params(clean_params)

    except Exception as e:
        print(f"Error while logging model parameters: {e}")
        raise

def array_to_df(arr):
    """Converts a numpy array back to a DataFrame using feature names from the encoder."""
    # get feature names from the fitted encoder_transformer
    # input_features must match the original column order (X.columns)
    feature_names = encoder_transformer.get_feature_names_out(input_features=X.columns)
    # build DataFrame. Use a simple RangeIndex to avoid index misalignment inside CV
    return pd.DataFrame(arr, columns=feature_names)

In [26]:
# Splitting Data
X = df.drop(columns=['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)
y = df['Machine failure']

assert isinstance(X, pd.DataFrame)

# Stratified Split to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=CONFIG["test_size"], 
    random_state=CONFIG["random_state"],
    stratify=y
)

In [27]:

total_combinations = len(scalers) * len(encoders) * len(samplers) * len(models)
print(f"Total combinations to test: {total_combinations}")

with mlflow.start_run(run_name="Preprocessing_and_Modeling") as parent_run:
    pbar = tqdm(total=total_combinations, desc="Pipeline combinations")
    
    for scaler_name, scaler in scalers.items():
        for encoder_name, encoder in encoders.items():
            for sampler_name, sampler in samplers.items():
                for algo_name, model in models.items():
                    combo_name = f"{scaler_name}_{encoder_name}_{sampler_name}_{algo_name}"
                    pbar.set_description(combo_name)
                    
                    with mlflow.start_run(run_name=combo_name, nested=True) as child_run:
                        try:
                            start_time = time.time()
                            
                            # --- A. Build Pipeline ---
                            encoder_transformer = ColumnTransformer(
                                transformers=[
                                    ('num', "passthrough", NUMERIC_FEATURES),
                                    ('cat', encoder, CATEGORICAL_FEATURES),
                                ])
                            
                            # Determine column indices after encoding
                            if encoder_name == "OneHotEncoder":
                                numeric_indices = list(range(8))
                                categorical_indices = list(range(8, 11))
                            elif encoder_name == "OrdinalEncoder":
                                numeric_indices = list(range(8))
                                categorical_indices = [8]
                            
                            scaler_transformer = ColumnTransformer(
                                transformers=[
                                    ('num', scaler, numeric_indices),
                                    ('cat', "passthrough", categorical_indices),
                                ]
                            )
                            
                            # Handle Class Weights
                            if sampler_name == "None":
                                try:
                                    model.set_params(class_weight='balanced')
                                except:
                                    pass
                            else:
                                # Reset class_weight if using a sampler
                                try:
                                    model.set_params(class_weight=None)
                                except:
                                    pass
                            
                            # Assembly
                            steps = [
                                ("eng", FeatureEngineer()),
                                ("encode", encoder_transformer),
                            ]
                            
                            if sampler is not None:
                                steps.append(("sampler", sampler))
                            
                            steps.append(("scale", scaler_transformer))
                            steps.append(('model', model))
                            
                            pipeline = ImbPipeline(steps=steps)
                            
                            # --- B. Evaluate (Cross-Validation) ---
                            cv_results = cross_validate(
                                pipeline, X_train, y_train,  
                                cv=StratifiedKFold(n_splits=5), 
                                scoring=['recall', 'f1', 'roc_auc'],
                                n_jobs=2,  # Reduce parallelization
                                verbose=0
                            )
                            
                            elapsed = time.time() - start_time
                            
                            # --- C. Log to MLflow ---
                            params = {
                                "encoder": encoder_name,
                                "imbalance_strategy": sampler_name if sampler is not None else "ClassWeight/None",
                                "scaler": scaler_name,
                                "model": algo_name
                            }
                            
                            mlflow.log_params(params=params)
                            log_model_params(algo_name=algo_name, model=model)
                            
                            # Logging Artifacts
                            mlflow.sklearn.log_model(pipeline, artifact_path="model")
                            
                            # Log Metrics
                            metrics = {
                                "test_recall_mean": cv_results['test_recall'].mean(),
                                "test_recall_std": cv_results['test_recall'].std(),
                                "test_f1_mean": cv_results['test_f1'].mean(),
                                "test_roc_auc_mean": cv_results['test_roc_auc'].mean(),
                                "cv_time_seconds": elapsed
                            }
                            
                            mlflow.log_metrics(metrics=metrics)
                            
                            # Logging Confusion Matrix
                            pipeline.fit(X, y)  # Fit on full training data
                            disp = ConfusionMatrixDisplay.from_estimator(pipeline, X_test, y_test)
                            plt.savefig("confusion_matrix.png")
                            mlflow.log_artifact("confusion_matrix.png")
                            plt.close()
                            
                            pbar.write(f"✓ {combo_name}: F1={cv_results['test_f1'].mean():.4f}, Recall={cv_results['test_recall'].mean():.4f}, Time={elapsed:.1f}s")
                        
                        except Exception as e:
                            pbar.write(f"✗ {combo_name}: ERROR - {str(e)}")
                            mlflow.log_param("status", "failed")
                            mlflow.log_param("error", str(e))
                        
                        finally:
                            pbar.update(1)
    
    pbar.close()

print("\n🎉 Training complete!")

Total combinations to test: 24


StandardScaler_OneHotEncoder_None_RandomForest:   8%|▊         | 2/24 [02:14<24:38, 67.20s/it]


Logging parameters for model: LogReg


d:\Conda\envs\smart-iiot\Lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\Conda\envs\smart-iiot\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(

                                                                             

✓ StandardScaler_OneHotEncoder_None_LogReg: F1=0.2784, Recall=0.8269, Time=0.1s


...

Logging parameters for model: RandomForest


d:\Conda\envs\smart-iiot\Lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\Conda\envs\smart-iiot\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(

                                                                                                        

✓ RobustScaler_OrdinalEncoder_RandomOverSampler_RandomForest: F1=0.8178, Recall=0.7490, Time=1.2s


RobustScaler_OrdinalEncoder_RandomOverSampler_RandomForest: 100%|██████████| 24/24 [06:41<00:00, 16.73s/it]



🎉 Training complete!



***

# 🔬 Phase 1.5 Report: Preprocessing & Feature Engineering Strategy

**Date:** December 09, 2025
**Author:** Prakash Dwivedi
**Module:** Predictive Maintenance (PdM)

---

![alt text](<Screenshot (118).png>)

## 1. Objective
The goal of this phase was to scientifically determine the optimal data preprocessing pipeline for the AI4I 2020 Predictive Maintenance dataset. Specifically, we aimed to answer:
1.  **Which Scaler** best handles the sensor outliers?
2.  **Which Encoder** works best for the categorical `Type` feature?
3.  **Which Imbalance Strategy** maximizes failure detection (Recall) without destroying precision?
4.  **Which Model Class** is suitable for Phase 2 development?

---

## 2. Methodology: The "Grid Search" Experiment
We conducted an exhaustive experiment logging **25 distinct pipeline configurations** to MLflow.
* **Scalers Tested:** `StandardScaler` vs. `RobustScaler`.
* **Encoders Tested:** `OrdinalEncoder` vs. `OneHotEncoder`.
* **Imbalance Handlers:** `None` (Class Weights) vs. `SMOTE` vs. `RandomOverSampler`.
* **Models:** `LogisticRegression` (Linear Baseline) vs. `RandomForest` (Non-linear Baseline).

**Metrics Tracked:**
* **Recall (Primary):** Critical for safety; we cannot miss failures.
* **F1-Score (Secondary):** To ensure we don't drown in false alarms.
* **ROC-AUC:** To measure overall separability.

---

## 3. Key Findings

### A. Model Selection: Tree-Based Dominance
* **Logistic Regression:** Failed completely.
    * **F1-Score:** ~0.28 (Unusable).
    * **Insight:** Linear models cannot capture the complex, non-linear interactions in this dataset (e.g., the inverse relationship between Torque and Speed, or the specific failure modes).
* **Random Forest:** Performed excellently.
    * **F1-Score:** > 0.80.
    * **Decision:** We will proceed with **Random Forest** (and likely XGBoost in Phase 2) as our core architecture.

### B. The "Recall vs. Precision" Trade-off
We observed a direct trade-off based on the imbalance strategy:
* **Class Weights (No Sampler):**
    * Recall: ~69% | F1: ~80%
    * *Result:* High precision, but misses ~31% of failures.
* **SMOTE (Synthetic Oversampling):**
    * Recall: ~79% | F1: ~76%
    * *Result:* ** Catches 10% more failures.** Slightly more false alarms.

**Strategic Decision:** We choose **SMOTE**. In predictive maintenance, a missed failure (False Negative) is catastrophic. We prioritize Recall and will manage the false positives later via probability threshold tuning.

### C. Encoding & Scaling
* **Encoder:** Surprisingly, **`OneHotEncoder`** performed best with Random Forest + SMOTE (Recall: 0.79 vs 0.78 for Ordinal). This also aligns with best practices for future Neural Network experiments.
* **Scaler:** **`RobustScaler`** showed stability and is theoretically superior for handling the Torque/Speed outliers identified in EDA.

---

## 4. Final Pipeline Configuration

Based on the evidence, the "Champion Pipeline" for Phase 2 is:

1.  **Feature Engineering:**
    * `Power` = Torque × Speed
    * `Temp_Diff` = Process Temp - Air Temp
    * `Wear_Status` = Binned Tool Wear
2.  **Preprocessing:**
    * **Scaling:** `RobustScaler` (on all numeric features).
    * **Encoding:** `OneHotEncoder` (on `Type` column).
3.  **Resampling (Training Only):**
    * **SMOTE** (Strategy: 0.5 or auto).
4.  **Model:**
    * **Random Forest Classifier** (Baseline for Phase 2 tuning).

---

